In [ ]:
#-----------------------------------------------------------------------
#  Import libs and Setup Environment
#-----------------------------------------------------------------------

import requests
import sys
import cv2
import pdb
import time
import pickle
import matplotlib
import os
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display

matplotlib.use('Agg')   #-- this is display-driver specific
dpi = 200 #-- change for your monitor/use case. why: https://stackoverflow.com/questions/13714454/specifying-and-saving-a-figure-with-exact-size-in-pixels

In [ ]:
#-----------------------------------------------------------------------
#  Define list of body dictionary objects, each with
#   - enabled: True | False, toggles processing of this body
#   - a friendly name for each body
#   - source map image url, from https://wiki.kerbalspaceprogram.com/wiki/Biome
#   - body radius in km, from each body's wiki entry
#   - a range of search radii min, max and step values:
#      - min: the minimum radius to search, expressed in percentage of body radius
#      - max: the maximum radius to search, expressd in percentage of body radius
#      - step: the percentage by which to increment between searches
#-----------------------------------------------------------------------

bodies = [
            {
                "enabled" : True,
                "friendly_name": "Moho",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/0/0f/Moho_Biome_Map_1.2.png",
                "radius_km": 250,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Eve",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/9/94/Eve_Biome_Map_1.2.png",
                "radius_km": 700,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Gilly",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/b/b1/Gilly_Biome_Map_1.2.1.png",
                "radius_km": 13,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Kerbin",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/b/b1/Kerbin_Biome_Map_1.2.png",
                "radius_km": 600,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            }, 
            {
                "enabled" : False,
                "friendly_name": "Mun",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/e/e0/Mun_Biome_Map_1.2.png",
                "radius_km": 200,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Minmus",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/f/f0/Minmus_Biome_Map_1.2.png",
                "radius_km": 60,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            }, 
            {
                "enabled" : False,
                "friendly_name": "Duna",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/a/a1/Duna_Biome_Map_1.2.png",
                "radius_km": 320,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Ike",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/b/b1/Ike_Biome_Map_1.2.png",
                "radius_km": 130,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            }, 
            {
                "enabled" : False,
                "friendly_name": "Dres",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/b/be/Dres_Biome_Map_1.2.png",
                "radius_km": 138,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Laythe",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/4/42/Laythe_Biome_Map_1.2.png",
                "radius_km": 500,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            }, 
            {
                "enabled" : False,
                "friendly_name": "Tylo",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/8/8a/Tylo_Biome_Map_1.2.png",
                "radius_km": 600,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Bop",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/a/a6/Bop_biome_1800x900.png",
                "radius_km": 65,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            },
            {
                "enabled" : False,
                "friendly_name": "Pol",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/0/05/Pol_Biome_Map_1.2.1.png",
                "radius_km": 44,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            }, 
            {
                "enabled" : False,
                "friendly_name": "Eeloo",
                "source_map_url": "https://wiki.kerbalspaceprogram.com/images/f/ff/Eeloo_Biome_Map_1.2.png",
                "radius_km": 210,
                "search_radii": {"min": .01, "max": 1.0, "step": .01}
            }
        ]

In [ ]:
#-----------------------------------------------------------------------
#  Download source maps, add local paths and filenames to body dicts
#-----------------------------------------------------------------------

for body in bodies:
    path = "./data/%s" % body['friendly_name']
    os.makedirs(path, exist_ok=True)
    url = body['source_map_url']
    fname = url.split('/')[-1]
    body['fname'] = fname
    body['fpath'] = "%s/%s" % (path,fname)
    print ('downloading: %s' % fname)
    resp = requests.get(url)
    with open((body['fpath']), 'wb') as f:
        f.write(resp.content)

In [ ]:
#----------------------------------------------------------------------
#  Review downloaded source maps
#----------------------------------------------------------------------

def get_image(f_path):
    image = open(f_path,'rb').read()
    return image

def tab_display(files, labels):
    images = [get_image(x) for x in files]
    children = [widgets.Image(value = img) for img in images if str(type(img)) != '<class \'NoneType\'>']

    # Customize layout:
    box_layout = widgets.Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        border='solid',
        width='50%')

    # Create the widget
    tab = widgets.Tab()
    tab.children = children

    # Label tabs
    for i in range(len(children)):
        tab.set_title(i,labels[i])

    display(tab)
    

#-- display original source maps
files = []
labels = []
for body in bodies:
    files.append(body['fpath'])
    labels.append(body['friendly_name'])
tab_display(files, labels)

In [ ]:
#-----------------------------------------------------------------------
#  DEFINE HELPER FUNCTIONS
#-----------------------------------------------------------------------

#-----------------------------------------------------------------------
#  Use your image library of choice to read in a color image.  Should
#  yield a (900,1800,3) numpy array of type uint8.
#-----------------------------------------------------------------------

def read_image(filename):
    image = cv2.imread(filename, flags=cv2.IMREAD_COLOR)
    return image

#-----------------------------------------------------------------------
#  Create the latitude, longitude coordinates for the center of each
#  pixel.  The image size (900x1800) suggests the intention is that
#  the pixels represents area and the edges of the pixels are lines of
#  longitude or latitude (e.g. the bottom _edge_ of the image is -90
#  latitude, but the _center_ of the pixels of the last row is -89.9
#  degrees South)
#-----------------------------------------------------------------------

def make_coordinates(height_pixels, width_pixels):
    
    delta_lat = 180.0 / height_pixels
    delta_lon = 360.0 / width_pixels

    lats = np.linspace(-90,90-delta_lat, height_pixels)+delta_lat/2
    lats = np.flip(lats, 0)   #-- to make first row north pole
    lons = np.linspace(-180,180-delta_lon, width_pixels)+delta_lon/2
    
    lon,lat = np.meshgrid(lons, lats)
    return lon,lat

#-----------------------------------------------------------------------
#  The image represents each biome with a distinct color triplet.
#  Here we convert the color image (HxWx3) into indexed biome number
#  (1,2,3,...), which makes downstream processing easier since we
#  don't have to keep track of a third dimension.
#-----------------------------------------------------------------------
def color_to_biome(img):
    height = img.shape[0]
    width = img.shape[1]

    biome_mapping = {}
    nbiomes = 0
    biome_array = np.zeros(height*width, dtype=np.uint8)

    #-- reshape the image to Nx3 and get the color triplet for each pixel
    for j,pixel in enumerate(np.reshape(img, (height*width,3))):
        triplet = tuple(pixel)
        if(triplet not in biome_mapping):
            #-- first time seeing this biome, so increment index
            nbiomes += 1
            biome_mapping[triplet] = nbiomes
        biome_array[j] = biome_mapping[triplet]

    return np.reshape(biome_array, (height,width))

#-----------------------------------------------------------------------
#   Calculate the great circle distance between two points on the
#   earth (specified in decimal degrees).  This is a vectorized
#   implementation: lons and lats may be numpy arrays, but lon1 and
#   lat1 should be scalar values.  There are likely some optimization
#   opportunities with this function.
#-----------------------------------------------------------------------

def haversine(lon1, lat1, lons, lats, body):

    R = body['radius_km']
    deg2rad = np.pi / 180.0
    
    dlon = (lons - lon1) * deg2rad
    dlat = (lats - lat1) * deg2rad
    
    a = np.sin(dlat/2)**2 + np.cos(lat1*deg2rad) * np.cos(lats*deg2rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(np.clip(a,-1,1)))
    d = c * R
    return d

In [ ]:
#-----------------------------------------------------------------------
#  Process each enabled body for optimal landing zones
#-----------------------------------------------------------------------
for body in bodies:
    if body['enabled']:
        print("####### processing %s" % (body['friendly_name']))
        os.chdir("./data/%s" % body['friendly_name'])
        filename = body['fname']
        planet = body['friendly_name']
        
        #-- display and save 1dimensional biome map, only need to do this once per body processed
        img = read_image(filename)
        height_pixels = img.shape[0]
        width_pixels = img.shape[1]

        #-- create arrays of lon,lat for pixel centers
        longitude, latitude = make_coordinates(height_pixels, width_pixels)

        #-- one dimensional arrays of lat and lon
        latitude_1d = latitude[:,0]
        longitude_1d = longitude[0,:]

        extent = (min(longitude_1d), max(longitude_1d), min(latitude_1d), max(latitude_1d))        
        biomes = color_to_biome(img)
        plt.imshow(biomes, extent=extent)
        plt.colorbar()
        pngname = '{}_biomes.png'.format(planet)
        plt.savefig(pngname, dpi=dpi)
        plt.show()
        
        for search_radius_pct in np.arange(body['search_radii']['min'], body['search_radii']['max'], body['search_radii']['step']):
            
            #-- set the radial distance being searched for this pass
            search_radius = body['radius_km'] * search_radius_pct #-- kilometers
            print("####### processing %s at search_radius %s km" % (planet, search_radius))

            #-- initialize empty output
            biome_count = np.zeros((height_pixels,width_pixels))

            #-------------------------------------------------------------------
            #  Exploit spherical symmetry: calculate the distance mask at a
            #  single latitude, then shift it horizontally (roll) to the
            #  various longitudes.  Avoids expensive recalculation of distance
            #  mask.  Still slower than it should be, I think the .roll
            #  function is badly implemented.  Bonus: it handles the date line
            #  branch cut automatically.
            #-------------------------------------------------------------------

            for i,lat1 in enumerate(latitude_1d):
                tic = time.time()

                #-- calculate mask once at this latitude
                distance = haversine(longitude_1d[0], lat1, longitude, latitude, body)
                mask = (distance <= search_radius)

                #-- step through each longitude and calculate diversity
                for j,lon1 in enumerate(longitude_1d):

                    #-- shift mask to match current longitude
                    m = np.roll(mask, j, axis=1)

                    #-- count the unique biomes within the specified distance of the current location
                    biome_count[i,j] = np.unique(biomes[m]).size

                toc=time.time()
                print("Done with latitude {:0.2f}:{}/{}, time elapsed {:0.6f}".format(lat1, i, height_pixels, toc-tic))

            os.makedirs('tmp', exist_ok=True)
            pklfile = './tmp/{}_diversity_{}_km.pkl'.format(planet, search_radius)
            with open(pklfile, 'wb') as fh:
                pickle.dump(biome_count, fh)

            #-- display functions
            plt.imshow(biome_count, extent=extent)
            plt.colorbar()

            #pdb.set_trace()

            pngname = './tmp/{}_diversity_{}_km.png'.format(planet, search_radius)
            plt.savefig(pngname, dpi=dpi)

            plt.show()
            
            #-- capture best score for this iteration at this search radius
            with open("%s.csv" % body['friendly_name'].lower(), 'a') as f:
                f.write("%s,%s,%s" % (search_radius, np.amax(biome_count), np.amax(biome_count)/search_radius))
                
        os.chdir('..')